In [1]:
import pandas as pd
import os

In [2]:
cleaned_EDF_folder = "../data/cleaned/EDF/"

tables = ["remuneration", "formation", "droit", "condition",
        "absenteisme", "effectif", "handicap", "exterieur"]

cleaned_dfs = {}

for table in tables:
    file_path = os.path.join(cleaned_EDF_folder, f"{table}.csv")
    cleaned_dfs[table] = pd.read_csv(file_path, sep=";")

In [3]:
def afficher_indicateurs_par_table(dfs, indicator_col="Indicateur"):
    """
    Affiche les noms des indicateurs présents dans la colonne `indicator_col` 
    pour chaque table du dictionnaire `dfs`.

    args:
        dfs (dict): Dictionnaire contenant les DataFrames (une par groupe).
        indicator_col (str): Nom de la colonne contenant les indicateurs.
    """
    for groupe, df in dfs.items():
        # Vérifie si la colonne des indicateurs existe dans la table
        if indicator_col not in df.columns:
            print(f"La colonne '{indicator_col}' est absente dans le groupe '{groupe}'")
            continue

        # Récupére les valeurs uniques dans la colonne "Indicateur"
        indicateurs = df[indicator_col].unique()
        
        print(f"\nGroupe: {groupe}")
        print("Indicateurs trouvés:")
        for indicateur in indicateurs:
            print(f"  - {indicateur}")


In [4]:
# Permet de créer le mapping des noms longs/courts des indicateurs

afficher_indicateurs_par_table(cleaned_dfs, indicator_col="Indicateur")


Groupe: remuneration
Indicateurs trouvés:
  - Promotions dans un collège supérieur
  - Masse salariale annuelle
  - Rémunération mensuelle moyenne brute

Groupe: formation
Indicateurs trouvés:
  - Montant consacré à la formation continue - INTERNE
  - Montant consacré à la formation continue - AUTRES
  - Salariés ayant suivi au moins un stage
  - Heures de formation
  - salariés ayant bénéficié d’un congé individuel de formation non rémunéré
  - Contrats d'apprentissage conclus dans l'année
  - Salariés ayant bénéficié d’un congé individuel de formation rémunéré
  - Pourcentage de l’assiette des rémunérations soumises aux cotisations de sécurité sociale du régime général afférent à la formation continue
  - Montant consacré à la formation continue - AGRÉÉ CONVENTION
  - Contrats de professionnalisation conclus dans l'année

Groupe: droit
Indicateurs trouvés:
  - Nombre de recours à des modes de solution non juridictionnels engagés dans l’année
  - Nombre d’instances judiciaires engagé

In [5]:
mapping_indicators = {
    "remuneration": {
        "Promotions dans un collège supérieur": "promo_college_sup",
        "Masse salariale annuelle": "masse_salariale_annuelle",
        "Rémunération mensuelle moyenne brute": "remu_mensuelle_brute"
    },
    "formation": {
        "Montant consacré à la formation continue - INTERNE": "mnt_fc_interne",
        "Montant consacré à la formation continue - AUTRES": "mnt_fc_autres",
        "Salariés ayant suivi au moins un stage": "sal_stage_au_moins_un",
        "Heures de formation": "heures_formation",
        "salariés ayant bénéficié d’un congé individuel de formation non rémunéré": "cif_non_remun",
        "Contrats d'apprentissage conclus dans l'année": "contrats_apprentissage",
        "Salariés ayant bénéficié d’un congé individuel de formation rémunéré": "cif_remun",
        "Pourcentage de l’assiette des rémunérations soumises aux cotisations de sécurité sociale du régime général afférent à la formation continue": "pct_assiette_formation",
        "Montant consacré à la formation continue - AGRÉÉ CONVENTION": "mnt_fc_agree_conv",
        "Contrats de professionnalisation conclus dans l'année": "contrats_pro"
    },
    "droit": {
        "Nombre de recours à des modes de solution non juridictionnels engagés dans l’année": "nb_recours_non_juridictionnels",
        "Nombre d’instances judiciaires engagées dans l’année où l’Entreprise\nest en cause (EDF et Mixtes)": "nb_instances_judiciaires"
    },
    "condition": {
        "Salariés bénéficiaires d'un système d'horaires individualisés": "horaires_individualises",
        "Salariés à temps partiel en décembre": "sal_temps_partiel_dec",
        "Salariés en réduction collective du temps de travail": "sal_reduc_collective_tt",
        "Salariés en services continus de plus de 50 ans": "sal_services_continus_50plus",
        "Salariés reclassés dans l'entreprise à la suite d'une inaptitude": "sal_reclasses_inaptitude",
        "Salariés ayant bénéficié de repos compensateur": "repos_compensateur",
        "Salariés déclarés définitivement inaptes à leur emploi par le médecin du travail": "sal_inaptes_medecin"
    },
    "absenteisme": {
        "Absence pour maladie": "abs_maladie",
        "Absence pour congé de maternité ou d'adoption": "abs_mat_adoption",
        "Absence pour congé de paternité et d'accueil de l'enfant": "abs_paternite",
        "Absence pour congés autorisés (événements familiaux, congés mères de famille, congés spéciaux…)": "abs_conges_autorises"
    },
    "effectif": {
        "Effectif": "effectif",
        "Embauches de salariés de moins de 25 ans": "embauches_moins25",
        "Démissions": "demissions"
    },
    "handicap": {
        "Salariés en situation de handicap": "sal_handicap",
        "Salariés reconnus travailleurs handicapés suite à accidents du travail survenus dans l'entreprise": "sal_handicap_at"
    },
    "exterieur": {
        "Stagiaires scolaires": "stagiaires_scolaires",
        "Salariés détachés en mobilité à l'international": "sal_detaches_mobilite_inter",
        "Salariés détachés accueillis": "sal_detaches_accueillis",
        "Nombre moyen mensuel de travailleurs temporaires": "nb_moyen_temp",
        "Durée moyenne des contrats de travail temporaire": "duree_moyenne_ctt"
    }
}

In [6]:
def generer_sous_tables(dfs, mapping_indicators, output_folder, indicator_col="Indicateur"):
    """
    Pour chaque table, on crée une sous-table par indicateur (nom long => nom court).
    On supprime ensuite les colonnes vides et on enregistre le fichier CSV.
    Pour chaque table de base (cleaned), on crée un dossier avec le même nom,
    puis dans chaque dossier, on met les indicateurs.
    """

    # Crée le répertoire de sortie s'il n'existe pas
    os.makedirs(output_folder, exist_ok=True)

    # On parcourt chaque table
    for table_name, df in dfs.items():
        # Vérifie si la table_name existe dans le mapping
        if table_name not in mapping_indicators:
            print(f"La table '{table_name}' n'est pas dans le mapping.")
            continue

        # Crée un sous-répertoire pour cette table s'il n'existe pas
        table_folder = os.path.join(output_folder, table_name)
        os.makedirs(table_folder, exist_ok=True)

        # Pour chaque indicateur (nom long) et son nom court
        for indicateur_long, indicateur_court in mapping_indicators[table_name].items():
            # Filtrage des lignes correspondant à cet indicateur
            sub_df = df[df[indicator_col] == indicateur_long].copy()

            if sub_df.empty:
                print(f"[WARNING] Aucune ligne pour '{indicateur_long}' dans la table '{table_name}'")
                continue

            # Supprime les colonnes entièrement vides
            sub_df.dropna(axis=1, how='all', inplace=True)

            # Chemin de sortie
            csv_filename = f"{indicateur_court}.csv"
            output_path = os.path.join(table_folder, csv_filename)

            # Enregistre en csv, écrase si le fichier existe déj
            sub_df.to_csv(output_path, index=False, sep=";")


In [7]:
output_folder = "../data/transformed/EDF/"
generer_sous_tables(cleaned_dfs, mapping_indicators, output_folder, indicator_col="Indicateur")